# 📖 Agent 實作範例：自動化有聲書生成

本 Notebook 展示如何透過 Agent 串接 **故事 → 圖像 → 語音** 的自動化流程。  
每個步驟都會將輸出保存為檔案，避免重複 Token 消耗，並方便後續流程使用。

---

## 🔹 整體流程
1. 故事內容生成  
2. 圖片內容生成  
3. 語音內容生成  

---

## 1️⃣ 故事內容生成
**輸入**  
- 草稿文字  
- 既有內容 (`.txt` 檔案)  

**處理流程**  
1. 將文字送入 Langserve 服務  
2. 接收生成的故事段落  

**輸出**  
- 將結果存為 `.txt` 檔  
- 避免 Agent 一直傳遞整份故事，降低 Token 消耗  

---

## 2️⃣ 圖片內容生成
**輸入**  
- 最新生成的故事文字  
- 既有圖片 (`.png`)  

**處理流程**  
1. 將圖片編碼為 base64  
2. 文字與圖片送入 Langserve 服務  
3. 接收回傳的圖片（base64 格式）  

**輸出**  
- 將 base64 解碼為二進位資料，存成 `.png` 檔  

---

## 3️⃣ 語音內容生成
**輸入**  
- 最新生成的故事文字  

**處理流程**  
1. 將文字送入 Langserve 服務  
2. 接收回傳的語音（base64 格式）  

**輸出**  
- 將 base64 解碼為二進位資料，存成 `.mp3` 或 `.wav` 檔  

---

## 🔄 流程圖

```mermaid
flowchart TD
    A[故事草稿/舊內容 .txt] --> B[送入 Langserve 生成故事]
    B --> C[故事內容 .txt]
    C --> D[送入 Langserve 生成圖片 (base64)]
    D --> E[解碼並存為 .png]
    C --> F[送入 Langserve 生成語音 (base64)]
    F --> G[解碼並存為 .mp3 / .wav]


## LangServe 服務測試

In [ ]:
import os
import requests

os.chdir("../../../")

測試故事生成服務

In [ ]:
response = requests.post(
    "http://localhost:8080/story_telling/invoke",
    json={"input":{'scratch': "Create a chapter of a baby owl capturing a rodent in the night as his dinner",
                   'context': ""}
         }
)

In [ ]:
story_json = response.json()

In [ ]:
story_json['output']

測試影像生成服務

In [ ]:
import importlib

image_generation_module = importlib.import_module("tutorial.LLM+Langchain.Week-8.logic.image_generation")
image_create_pipeline = image_generation_module.image_create_pipeline(image_generation_module.system_template)

In [ ]:
response_image = requests.post(
    "http://localhost:8080/image_generation/invoke",
    json={"input": {'story': story_json['output'],
                    "image_io": []}
    }
)

In [ ]:
response_image.json()['output']['nl_prompt']

In [ ]:
import base64
# Decode to bytes
image_bytes = base64.b64decode(response_image.json()['output']['image_base64'])

with open("tutorial/LLM+Langchain/Week-8/story_2_image.png", "wb") as fh:
    fh.write(image_bytes)

測試生成後續後續的故事

In [ ]:
response_next_tory = requests.post(
    "http://localhost:8080/story_telling/invoke",
    json={"input":{'scratch': "Create the next chapter following the context",
                   'context': story_json['output']}
         }
)

In [ ]:
next_chapter = response_next_tory.json()['output']

根據故事和上一張圖片，產生出下一張圖片

透過requests送出base64 string

In [ ]:
response = requests.post(
    "http://localhost:8080/image_generation/invoke",
    json={"input": {'story': next_chapter + f"\nPrevious image description:\n\n{response_image.json()['output']['nl_prompt']}",
                    # 'image_io': [response_image.json()['output']['image_base64']]
                    'image_io': []
                   }
    }
)

In [ ]:
response.json()['output']['nl_prompt']

In [ ]:
import base64

image_bytes = base64.b64decode(response.json()['output']['image_base64'])

with open("tutorial/LLM+Langchain/Week-8/story_3_image.png", "wb") as fh:
    fh.write(image_bytes)

測試語音生成

In [ ]:
response = requests.post(
    "http://localhost:8080/audio_generation/invoke",
    json={"input": {'input': "How are you doing?"}}
)

In [ ]:
audio_bytes = base64.b64decode(response.json()['output'])

with open("tutorial/LLM+Langchain/Week-8/test_sample.mp3", "wb") as f:
    f.write(audio_bytes)


## 生成工具模板

In [ ]:
from textwrap import dedent
from typing import Optional, Any, List, Tuple, Callable
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.tools import BaseTool
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field
from pydantic import FilePath

from src.initialization import credential_init


class ToolTemplate(BaseTool):

    """
    ToolTemplateHTTP: 一個專門用來呼叫 Langserve REST API 的 Agent Tool

    - 使用 PydanticOutputParser 保證輸入格式正確
    - 支援多欄位 input/output 處理器
    - 對 API 呼叫加上錯誤處理
    """
    
    runnable: str = Field(..., description='The Langserve endpoint')
    name: str
    input_parser: PydanticOutputParser
    description: str
    input_data_processors: Optional[List[Tuple[str, Callable[[Any], Any]]]] = None
    output_data_processors: Optional[List[Tuple[Optional[str], Callable[[Any], Any]]]] = None
    
    @classmethod
    def create(cls, runnable: str, name: str, description_template: str,
               input_parser: PydanticOutputParser, input_data_processors: Optional=None,
               output_data_processors: Optional=None):

        """建立 Tool 實例，會自動把輸入格式需求加入 description"""
        
        input_format_instruction = input_parser.get_format_instructions()
        
        description = description_template.format(
            input_format_instruction=input_format_instruction
        )
        
        return cls(runnable=runnable, name=name, description=description,
                   input_parser=input_parser, input_data_processors=input_data_processors,
                   output_data_processors=output_data_processors)
    
    def _run(self, query: str):

        """執行 Tool，同步版本"""
        
        # 1. 驗證 & parse 輸入
        try:
            input_ = self.input_parser.parse(query)
        except Exception as e:
            raise ValueError(f"Failed to parse input with parser: {e}, query={query}")
        
        runnable_inputs = input_.model_dump()

        # 2. input processors（前處理）
        if self.input_data_processors:
            for field, fn in self.input_data_processors:
                if field in runnable_inputs:
                    runnable_inputs[field] = fn(runnable_inputs[field])
            
        # 3. 呼叫 Langserve REST API
        try:
            response = requests.post(
                str(self.runnable),
                json={"input": runnable_inputs},
                timeout=60,
            )
            response.raise_for_status()
            result = response.json()
        except Exception as e:
            raise RuntimeError(f"Langserve call failed: {e}, inputs={runnable_inputs}")

        if "output" not in result:
            raise RuntimeError(f"Invalid response format from Langserve: {result}")
        
        output = result['output']

        # 4. update the state varaibles:
        for key in session_state.keys():
            if key in output:
                session_state[key] = output[key]
        
        # 5. output processors（後處理）
        if self.output_data_processors:
            for field, fn in self.output_data_processors:
                if not field:
                    fn(output, runnable_inputs['filename'])
                else:
                    fn(output[field], runnable_inputs['filename'])
                    
        # 預設回傳「檔名」如果有 filename，否則回傳輸出的字串
        return runnable_inputs.get("filename", output)

    def _arun(self, query: str):

        raise NotImplementedError("This tool does not support async")

### State Variables

In [ ]:
session_state = {}

session_state['nl_prompt'] = None

In [ ]:
from pathlib import Path

from pydantic import BaseModel, Field


class StoryInput(BaseModel):
    scratch: str = Field(description=dedent("""\
                                            The draft, notes, or rough idea for the current page of the story.
                                           （故事當前頁面的草稿、筆記或初步構想
                                            """))
    context: List[FilePath] = Field(default_factory=list, description=dedent("""\
                                                              A list of previously generated .txt files that contain story content.  
                                                              Used to maintain narrative consistency and continuity across images.  
                                                              先前生成的 .txt 檔案清單，其中包含故事內容。  
                                                              用於保持影像生成過程中的敘事一致性與連貫性。
                                                              """))
    filename: str = Field(..., description=dedent("""\
                                             The file path where the generated story text will be saved.
                                            （生成的故事文本將被儲存的檔案路徑）
                                             """))


def export_to_txt(text, filename: Path):

    dir_ = Path(filename).parent

    if not os.path.isdir(dir_):
        os.makedirs(dir_)
    
    with open(filename, "w", encoding="utf-8") as file:
        file.write(text)


def read_from_txt(filename) -> str:

    with open(filename, "r", encoding="utf-8") as file:
        content = file.read()

    return content


def read_from_list_of_text(filenames) -> str:

    return "\n\n".join([read_from_txt(f) for f in filenames])


story_input_data_processors = [("context", read_from_list_of_text)]

story_output_data_processors = [(None, export_to_txt)]

story_telling_tool = ToolTemplate.create(
    runnable="http://localhost:8080/story_telling/invoke",
    name="Story generation tool",
    description_template=dedent("""\
                                This tool is designed to generate a story one page at a time.
                                Provide a draft or idea for the current page (`scratch`), along with 
                                the preceding story context stored as .txt files (`context`), 
                                and specify where the generated text should be saved (`filename`).
                                    
                                Input format: {input_format_instruction}
                                """
                                ),
    input_parser=PydanticOutputParser(pydantic_object=StoryInput),
    output_data_processors = story_output_data_processors,
    input_data_processors = story_input_data_processors
)

### 影像生成

In [ ]:
class ImageInput(BaseModel):
    story: str = Field(description=dedent("""\
                                          The narrative or context used to generate the image prompt.  
                                          故事情節或上下文，用於生成影像提示。
                                          """)
                      )
    # FilePath ensures the input path exists and is a valid file
    image_io: List[str] = Field([], description=dedent("""\
                                                   Path to the previously generated image.  
                                                   Used in img2img generation to maintain visual and texture consistency.  
                                                   先前生成影像的路徑。  
                                                   在 img2img 生成中用於保持視覺與材質的一致性。
                                                   """)
                                    )
    # Path allows flexibility: file may not exist yet but must be a valid path object.
    filename: str = Field(..., description=dedent("""\
                                                  Destination file path where the generated image will be saved.  
                                                  生成影像的儲存檔案路徑。
                                                  """)
                          )


def image_to_base64(image_path):
    
    with Image.open(image_path) as image:
        
        # Save the Image to a Buffer
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        
        # Encode the Image to Base64
        image_str = base64.b64encode(buffered.getvalue())
    
    return image_str.decode('utf-8')


def image_to_base64_from_list(filenames) -> List[Optional[str]]:

    # return [image_to_base64(f) for f in filenames]
    return []


def export_to_image(content, filename):
    
    dir_ = Path(filename).parent

    if not os.path.isdir(dir_):
        os.makedirs(dir_)
    
    image_bytes = base64.b64decode(content)
    
    with open(filename, "wb") as fh:
        fh.write(image_bytes)


def story_adaptation(story: str):

    nl_prompt = session_state['nl_prompt']
    
    if nl_prompt:
        story += f"\nPrevious image description:\n\n{nl_prompt}"

    print(f"******\n{story}\n*******")
    
    return story
    


image_output_data_processors = [('image_base64', export_to_image)]

image_input_data_processors = [("story", story_adaptation),
                                ("image_io", image_to_base64_from_list)]

image_tool = ToolTemplate.create(
    runnable="http://localhost:8080/image_generation/invoke",
    name="Image generation tool",
    description_template=dedent("""\
                                This tool is designed to generate an image to the correspoinding narrative.
                                Provide a narrative (`story`), along with 
                                the preceding images stored as .png files (`image_io`), 
                                and specify where the generated text should be saved (`filename`).
                                    
                                Input format: {input_format_instruction}
                                """
                                ),
    input_parser=PydanticOutputParser(pydantic_object=ImageInput),
    output_data_processors = image_output_data_processors,
    input_data_processors = image_input_data_processors
)

### 語音生成

In [ ]:
class AudioInput(BaseModel):
    input: str = Field(description=dedent("""\
                                          The narrative or context used to generate the audio content with text to sound (TTS).  
                                          故事情節或上下文，用於TTS文字轉語音。
                                          """)
                                    )
    # Path allows flexibility: file may not exist yet but must be a valid path object.
    filename: str = Field(..., description=dedent("""\
                                                  Destination mp3 file path where the generated audio will be saved.  
                                                  mp3語音檔的儲存檔案路徑。
                                                  """)
                         )

def export_to_audio(content, filename):

    dir_ = Path(filename).parent

    if not os.path.isdir(dir_):
        os.makedirs(dir_)
    
    audio_bytes = base64.b64decode(content)
    
    with open(filename, "wb") as fh:
        fh.write(audio_bytes)


audio_output_data_processors = [(None, export_to_audio)]

audio_tool = ToolTemplate.create(
    runnable="http://localhost:8080/audio_generation/invoke",
    name="Audio generation tool",
    description_template=dedent("""\
                                This tool is designed to generate an .mp3 file to a corresponding narrative.
                                Provide a narrative (`story`), along with 
                                and specify where the generated audio should be saved (`filename`).
                                    
                                Input format: {input_format_instruction}
                                """
                                ),
    input_parser = PydanticOutputParser(pydantic_object=AudioInput),
    output_data_processors = audio_output_data_processors,
)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

tools = [story_telling_tool, image_tool, audio_tool]

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o", temperature=0, 
                  )

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
query = dedent("""
Create a chapter of a baby owl capturing a rodent in the night as his dinner.
After having the final answer, please create a corresponding image and a corresponding mp3 file.
The saved image (.png), text (.txt), and audio (.mp3) should have same name in the folder `tutorial/LLM+Langchain/Week-8/story_test`
""")

agent_executor.invoke({"input": query})

In [ ]:
session_state

成功的生成了一頁的內容，Agent可以幫我們生成整個故事嗎?

In [ ]:
prompt = """
         I want to create an 4 pages story for a child. He likes snow owl.
         For each page, please create a corresponding image and record the story as an mp3.
         After having the final answer, please create a corresponding image and record the story as an mp3. 
         The saved image and mp3 should have same name, following the structure of 
         <Page - idx>, with idx as a number starting from 1, in the folder `tutorial/LLM+Langchain/Week-8/story_automation`
         """

agent_executor.invoke({"input": prompt})

## 🎧 互動式有聲書內容生成

- 不一定需要完整的 **Agent 架構**，因為流程的每一步（故事 → 圖像 → 語音）都已經明確定義，能由使用者主動觸發。  
- 可以直接基於 **聊天機器人** 的互動形式進行，每次輸入使用者的需求或指令後，系統依照指定步驟生成對應內容。  
- 使用者可以在故事生成過程中即時調整方向，例如指定角色、情節走向或語氣，提升 **客製化體驗**。  
- 這種互動方式非常適合 **語言學習** 場景：  
  - 學習者能一邊閱讀故事、一邊聽有聲輸出  
  - 可即時修改故事情節，產生更貼近學習需求的內容  
  - 搭配圖片與語音，提升沉浸式學習效果  

# Langgraph

在現代 LLM 應用開發中，**LangChain** 和 **LangGraph** 是兩個常被提及的工具，它們皆旨在簡化與大型語言模型的整合流程，但各自的設計理念和使用場景略有不同。

## LangChain vs. LangGraph 比較

| 特點 | LangChain | LangGraph |
|------|-----------|-----------|
| 架構風格 | 函式導向（Function-based） | 圖形導向（Graph-based） |
| 核心用途 | 組合不同的工具與鏈（Chains）來處理自然語言任務 | 定義狀態轉換與流程控制的狀態機 |
| 適合場景 | 線性流程、多工具串接、Agent任務 | 有狀態的工作流程、多步驟決策、動態流程 |
| 控制流程能力 | 較弱，需透過程式邏輯控制 | 較強，內建狀態管理與動態流程切換 |
| 易學程度 | 相對容易上手 | 初期需要理解狀態機模型 |
| 通用性 | 通用性強、模組多 | 更適合複雜與長期任務管理 |

## 為什麼選擇 LangGraph？

LangGraph 建立在 LangChain 的基礎上，但提供了更明確的流程控制能力。透過「有向圖」的方式定義節點（Node）與狀態轉移（Edge），開發者可以更容易地設計出具備**記憶狀態**、**可回溯性**、**動態流程切換**的 LLM 應用。

簡而言之，**LangChain** 更像是設計工具組合的「積木箱」，而 **LangGraph** 則是設計流程和邏輯的「流程編輯器」。

---

接下來，我們將透過實作一個 **計劃-行動代理人（Plan-Action Agent）** 的例子，深入探索 LangGraph 的設計理念與強大功能。


## 什麼是計劃-行動代理人？

計劃-行動代理人（Plan-Action Agent）是智能體的一種，它依賴於計劃生成和行動執行來達成目標。簡單來說，這種代理人會首先根據當前情況建立一個行動計劃，然後根據計劃逐步執行各項任務，直到達成預定的目標。

計劃-行動代理人的工作流程通常包含以下幾個步驟：
1. **觀察環境**：代理人會先收集當前的環境狀態信息。
2. **計劃生成**：根據當前狀況，代理人會生成一個包含一系列行動的計劃。
3. **行動執行**：代理人按照計劃中的步驟逐一執行，並在必要時調整計劃。
4. **反饋與調整**：根據執行結果和環境變化，代理人會更新計劃，確保最終目標的達成。

## Langgraph 中的計劃-行動代理人

Langgraph 是一個強大的框架，它允許我們定義和實現多種形式的智能代理系統。在 Langgraph 中，計劃-行動代理人通常會利用圖論和計劃生成算法來選擇最佳的行動方案。通過簡單的接口和模塊化設計，Langgraph 使得構建和訓練這類代理人變得更加高效且可擴展。

在接下來的教程中，我們將展示如何使用 Langgraph 構建和訓練一個計劃-行動代理人，並通過一些實際範例來說明其應用場景。

In [ ]:
import os

os.chdir("../../../")

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", 
                   temperature=0
                  )

In [ ]:
import operator
from typing import Annotated, List, Tuple, Union, Optional
from typing_extensions import TypedDict

# The State

class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: List[Tuple]
    response: str

In [ ]:
from typing import List, Literal, Union
from textwrap import dedent
from pydantic import BaseModel, Field

from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate


class Plan(BaseModel):
    """
    Plan to follow in the future
    """
    type: Literal['plan'] = 'plan'
    steps: List[str] = Field(description="different steps to follow, should be in sorted order")


planner_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        dedent("""\
        For the given objective, come up with a simple step by step plan.
        This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps.
        The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.
        """)),
        ("placeholder", "{messages}")
    ]
)

model_planner = model.with_structured_output(Plan)

In [ ]:
planner = planner_prompt|model_planner

In [ ]:
planner.invoke(
    {
        "messages": [
            ("user", "What is the hometown of the current Australia open winner")
        ]
    }
)

In [ ]:
# class Response(BaseModel):
#     """Response to user."""
#     type: Literal["response"] = "response"
#     response: str = Field(descirption="The answer to the objective")


# class Act(BaseModel):
#     """Action to Perform."""

#     response = Optional[str] = Field(None, description="Direct response to the user.")
#     steps = Optional[List[str]] = Field(None, description="Plan steps to follow, in sorted order")

class Response(BaseModel):
    """Response to user."""
    type: Literal['response'] = 'response'
    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description=dedent("""\
        Action to perform. If you want to respond to user, use `response`.
        If you need to further use tools to get the answer, use `plan`.
        """
    )
    )

replanner_prompt = ChatPromptTemplate.from_template(
    dedet("""For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
))

model_act = model.with_structured_output(Act)

replanner = replanner_prompt | model_act

In [ ]:
replanner.invoke({"plan": "Identify the current Australian Open winner for the most recent tournament.",
                  "past_steps": [],
                  "input": "What is the hometown of the current Australia open winner"})

In [ ]:
!pip install langgraph

In [ ]:
input_parser = PydanticOutputParser(pydantic_object=Inputs)

In [ ]:
print(input_parser.get_format_instructions())

In [ ]:
import asyncio

from openai import OpenAI
from langchain.tools import BaseTool
from langchain.output_parsers import PydanticOutputParser

client = OpenAI()


class Inputs(BaseModel):
    query: str = Field(description="User query")


class SearchTool(BaseTool):

    input_parser: PydanticOutputParser = PydanticOutputParser(pydantic_object=Inputs)
    input_format_instructions: str = input_parser.get_format_instructions()
    
    name:str = "websearch tool"
    description_template: str = dedent("""
    Currently it is 2025.    
    Use this tool to collect information from the internet, when you are not sure you know the answer.
    Action Input format instructions: {input_format_instructions}
    """)

    description: str = description_template.format(input_format_instructions=input_format_instructions)
    
    def _run(self, query):
        
        input_ = self.input_parser.parse(query)
        
        query = input_.query
        
        messages = [{"role": "user",
                     "content": query}]

        response = client.chat.completions.create(
            model='gpt-4o-mini-search-preview',
            web_search_options={"search_context_size": 'medium',
                                "user_location": {
                                        "type": "approximate",
                                        "approximate": {
                                            "country": "TW",
                                        }
                                    },
                                },
            messages=messages
        )
        
        return response.choices[0].message.content
    
    async def _arun(self, query: str):
        """Asynchronous version of the run method."""
        input_ = self.input_parser.parse(query)
        query = input_.query
        
        messages = [{"role": "user", "content": query}]
        
        # Run the synchronous API call in an asynchronous way using asyncio.to_thread
        response = await asyncio.to_thread(
            client.chat.completions.create,
            model='gpt-4o-mini-search-preview',
            web_search_options={
                "search_context_size": 'medium',
                "user_location": {
                    "type": "approximate",
                    "approximate": {
                        "country": "TW",
                    }
                },
            },
            messages=messages
        )

        return response.choices[0].message.content

In [ ]:
agent_template = dedent("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Previous conversation history:

Question: {input}

Thought:{agent_scratchpad}
"""
)

agent_prompt_template = PromptTemplate.from_template(agent_template)

In [ ]:
from typing import Literal

from langgraph.graph import END
from langchain.agents import AgentExecutor, create_react_agent
# from langgraph.prebuilt import create_react_agent


tools = [SearchTool()]
agent_model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o", temperature=0)


# Choose the LLM that will drive the agent
zero_shot_agent = create_react_agent(llm=agent_model, tools=tools, prompt=agent_prompt_template)
agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i + 1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    
    agent_response = await agent_executor.ainvoke(
        {"input": task_formatted}
    )

    current_step = [(task, agent_response['output'])]
    
    if "past_steps" not in state:
        return {"past_steps": current_step}
    else:
        return {"past_steps": state["past_steps"] + current_step}


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    print("*************************")
    print(output)
    print("*************************")
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        if output.action.steps:
            return {"plan": output.action.steps}
        else:
            return {"response": state['past_steps'][-1][1]}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    should_end,
    ["agent", END]
)

app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"recursion_limit": 50}
inputs = {"input": "what is the hometown of the mens 2023 Australia open winner?"}

In [ ]:
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)